# Elastic Network Interface 고급
- ENI : 탄력적 네트워크 인터페이스는 VPC에서 가상 네트워크 카드를 나타내는 논리적 네트워킹 구성 요소입니다
- EC2의 가상 랜카드
    - IP 주소와 MAC 주소를 보유
    - ENI 하나 당 Private IP + 하나의 Public IP(Optional)
    - 필요에 따라서 한 개 이상의 Private IP 부여 가능
- EC2는 반드시 하나 이상의 ENI가 연결되어 있음
    - 제일 처음 EC2를 생성할 때 Primary ENI가 생성되어 연결됨
    - 추가로 ENI 연결 가능 -> 즉 EC2는 하나 이상의 ENI 보유 가능
    - 추가적인 ENI의 경우 EC2와 같은 가용영역이면 다른 서브넷에도 설정 가능
- 실질적으로 EC2의 서브넷 위치, 보안그룹 등 외부 연결과 관련될 설정은 ENI에서 결정

## 다중 ENI 아키텍처
- 하나의 EC2 인스턴스에 여러 ENI를 연동 가능
- 사용 사례
    - ENI 교체를 통한 배포/업데이트
    - 관리를 위하여 하나의 EC2 인스턴스에 다양한 접근 경로 설정
    - MAC 어드레스에 종속된 라이선스 프로그램을 다양한 EC2에서 사용
- 동시에 연동 가능한 ENI 숫자는 EC2의 타입과 크기에 따라 다름

## ENI와 보안그룹
- 보안그룹 적용은 ENI 단위
    - 즉 하나의 EC2 인스턴스에 다양한 보안 그룹으로 구성된 경로를 적용 가능
    - 예 : Subnet A에서는 80번만 허용, Subnet B에서는 22번만 허용
- 참고로 NACL은 Subnet 단위

## EC2와 Public IP
- EC2 Public IP는 ENI와 Public IP <-> Private IP 1:1 매칭(NAT)
    - 이 레코드는 Elastic IP로 고정하지 않는 이상 영구적인 레코드가 아님
    - EC2의 중지 -> 재부팅 시 Private IP는 바뀌지 않으나, Public IP는 변경(IP 테이블에서 퍼블릭 아이피가 삭제됨)
- 인터넷에서 Public IP로 통신이 전달되면 IGW가 Static NAT를 통해 변환 후 전달
- 즉 EC2의 OS는 절대로 Public IP를 알 수 없음
    - 즉 어떤 문제도 EC2 내부에서 Public IP를 설정해서 풀 수 없음
    - Private IP의 경우 OS에서 확인 가능
- EC2 생성할 떄 만들어지는 Primary ENI가 아닌 경우에 Public IP를 부여하려면 Elastic IP의 활용 필요

## Source/Destination Check
- ENI는 기본적으로 자신이 발생시켰거나, 자신이 대상이 아닌 트래픽은 무시
- 단 설정에 따라서 해제 가능
    - NAT Instance 등 자신을 위한 트래픽이 아닌 다른 대상에게 중계해주는 경우 해제 필요
- ENI 단위

## ENA vs EFA
- ENA(Elastic Network Adapter) : EC2의 네트워킹 속도를 최대 100Gbps까지 향상 가능
    - 낮은 Latency, 높은 I/O
    - 지원하는 EC2 인스턴스만 사용 가능 (Nitro-Based)
- EFA(Elastic Fabric Adapter) : 주로 AI, ML, HPC등을 위한 퍼포먼스를 지원하는 어댑터
    - 엄청나게 낮은 Latency, 높은 Throughtput 지원
    - 주로 매우 높은 사양의 EC2 인스턴스만 지원 (최소 12xlarge, 보통 24xlarge 이상)

### Demo - Multi ENI 아키텍처
1. 보안 그룹 2개 생성
    - 하나는 80번 http 포트, Name 태그 입력 (web)
    - 하나는 22번 ssh 포트, Name 태그 입력 (ssh)
2. 인스턴스 생성
    - ssh 보안그룹 적용
    - 유저데이터 입력
3. 탄력적 IP 생성, ENI 추가
    - 탄력적 IP를 ENI에 할당
4. ENI를 EC2 인스턴스와 연결
    - 인스턴스의 프라이빗 Ip가 두개가 된 것을 확인
    - 탄력적 IP로 접속해보자
5. 리소스 정리 : 인스턴스, EIP, ENI